# 10. 그래프 이론

## 실전 문제

### 팀 결성 // RE(틀림)

#### 제한
* 풀이 시간 20분
* 시간 제한 2초
* 메모리 제한 128MB

##### 아이디어
* 서로소 집합 알고리즘의 find, union 기능을 함수로 만들어서 사용하자.

#### 시간 복잡도
* $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 라고 한다. 정확한 계산은 조금 복잡할 듯 하다.
    * [참고할 만한 링크](https://hazel-developer.tistory.com/272)

#### 해설 본 후
* N, M의 범위가 모두 최대 100,000이므로 경로 압축을 이용한 서로소 집합 자료구조를 활용해야 함.
    * 경로 압축을 이용하지 않으면 시간 복잡도가 O(NM)이고, 이용하면 $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 이므로 이 경우엔 경로 압축을 해야 시간 내에 해결 가능
* 기왕이면 가독성을 위해 함수를 앞에다 설정하고, 함수를 앞에다 설정하기 위해 팀을 기록할 리스트를 파라미터에 추가하자.
* 함수들을 잘못 짰다. find를 만들었으면 union에 이를 사용해야 하는데 그렇게 하지 않아 경로 압축이 안 됐을 것이다. 테스트 케이스에는 운좋게 통과가 되었다.
* 같은 팀 여부 확인 연산을 굳이 함수로 만들 필요 없이, find를 두 번 이용하여 바로 비교할 수 있다.

In [2]:
# 학생 번호 N, 연산 개수 M 입력
N, M = map(int, input().split())

# 학생들의 팀을 기록할 리스트 선언(자기 자신을 팀으로 초기화)
teams = [i for i in range(N+1)]

# 해당 학생의 팀을 찾는 함수 정의
def find_team(a):
    if teams[a] != a:
        teams[a] = find_team(teams[a])
    return teams[a]

# 팀 합치기 연산 정의
def union_team(a, b):
    team_a = teams[a]
    team_b = teams[b]
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b
    
# 같은 팀 여부 확인 연산 정의
def check_same_team(a, b):
    if teams[a] == teams[b]:
        return 'YES'
    else:
        return 'NO'

# 연산 조건 입력 받아서 진행
for _ in range(M):
    oper, a, b = map(int, input().split())
    
    # 합치기 연산
    if oper == 0:
        union_team(a, b)
    # 같은 팀 여부 확인 연산
    else:
        print(check_same_team(a, b))

NO
NO
YES


#### 해설 코드 참조

In [4]:
# 특정 원소가 속한 집합을 찾기
def find_team(teams, a):
    # 루트 노드가 아니라면, 루트 노드를 찾을 때까지 재귀적으로 호출
    if teams[a] != a:
        teams[a] = find_team(teams, teams[a])
    return teams[a]

# 두 원소가 속한 집합을 합치기
def union_team(teams, a, b):
    team_a = find_team(teams, a)
    team_b = find_team(teams, b)
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b

N, M = map(int, input().split())
teams = [i for i in range(N+1)] # 부모 테이블 초기화(부모를 자기 자신으로)
    
# 각 연산을 하나씩 확인
for _ in range(M):
    oper, a, b = map(int, input().split())
    # 합집합(union) 연산인 경우
    if oper == 0:
        union_team(teams, a, b)
    # 찾기(find) 연산인 경우
    elif oper == 1:
        if find_team(teams, a) == find_team(teams, b):
            print('YES')
        else:
            print('NO')

NO
NO
YES


### 도시 분할 계획
https://www.acmicpc.net/problem/1647

#### 제한
* 풀이 시간 40분
* 시간 제한 2초
* 메모리 제한 256MB

#### 아이디어
* 크루스칼 이용하여 최소 신장 트리 만든 다음, 가장 높은 비용의 간선 하나를 제거하면 될 것 같다.
* 시간 초과가 난다. 입력값이 최대 1,000,000개가 될 수 있으므로 sys.stdin.readline()을 이용해보자 => 됐다.

#### 시간 복잡도
* 크루스칼 알고리즘을 사용했고, 간선의 개수가 M개이므로 O(MlogM).
* M은 최대 1,000,000이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 어차피 간선이 유지비 기준 오름차순으로 정렬되어 있으므로, 최대 유지비의 길을 저장할 때 max()를 쓸 필요 없이 해당 값으로 갱신만 해주면 된다.
* 답지에선 sys.stdin.readline()을 안 쓰고 그냥 input()으로 했는데, 백준에서 채점 돌려보면 input()으로 하면 시간초과가 난다.

In [14]:
import sys

# 어떤 집이 속한 마을을 확인하는 함수 선언
def find_group(groups, a):
    if groups[a] != a:
        groups[a] = find_group(groups, groups[a])
    return groups[a]

# 두 집이 속한 마을을 같은 마을로 합치는 함수 선언
def union_group(groups, a, b):
    group_a = find_group(groups, a)
    group_b = find_group(groups, b)
    
    if group_a < group_b:
        groups[group_b] = group_a
    else:
        groups[group_a] = group_b

# 집 개수 N, 길 개수 M 입력
N, M = map(int, input().split())

# 집들이 속한 마을 정보를 담는 리스트 선언(자신이 속한 마을을 자신의 마을로 초기화)
groups = [i for i in range(N+1)]

# 길의 정보 입력
roads = []
for _ in range(M):
    A, B, C = map(int, sys.stdin.readline().split())
    # A번 집에서 B번 집으로 가는 경로의 유지비가 C임.
    roads.append((C, (A, B)))

# 길을 유지비 기준 오름차순으로 정렬
roads.sort()

# 최소 신장 트리를 통해 구한 길의 전체 유지비와 길 중 가장 유지비가 높은 길의 유지비를 초기화
result = 0
max_cost = 0

# 모든 길을 확인하며
for cost, road in roads:
    # 길로 이어진 두 집의 속한 마을이 다르다면
    if find_group(groups, road[0]) != find_group(groups, road[1]):
        # 속한 마을을 합쳐주고
        union_group(groups, road[0], road[1])
        # 전체 유지비에 해당 길의 유지비를 추가하고
        result += cost
        # 지금까지 확인한 길보다 유지비가 비싸다면 최대 유지비를 가진 길을 갱신
        max_cost = max(max_cost, cost) ## 어차피 오름차순 정렬되어 있으므로 max()를 쓰지 않고 갱신해도 된다.

# 전체 유지비에서 가장 유지비가 큰 길을 빼서 마을 두 개를 생성할 수 있다.
print(result - max_cost)

8


### 커리큘럼 // RE(해결 못 함)

#### 제한
* 풀이 시간 50분
* 시간 제한 2초
* 메모리 제한 128MB

#### 아이디어
* 선수 강의의 개수가 적은 것부터 확인한다.
* 어떤 강의의 선수 강의 중 indegree가 같은 레벨에 있는 강의가 여러 개라면 그 중에 가장 시간이 오래 걸리는 것만 시간에 추가한다.
* 어떤 강의의 시간을 체크할 때 선수 강의 목록에 선수 강의와 선수 강의의 선수 강의가 함께 들어있다면, 후자의 시간을 중복 체크하지 않도록 한다.

* 근데 여기선 상위 강의의 정보에 선수 강의가 들어가 있어서 거꾸로 봐야할 것 같은데... => 방향을 뒤집어서 기본 코드와 비슷하게 가보자

#### 시간 복잡도
* 해설 코드 기준 최악의 경우엔 O(N!)이 되지 않나? N번 강의가 1번부터 N-1번까지를 모두 선수 과목으로 한다면 간선이 그렇게 될 것 같은데
    * 아니구나. 노드 N개일 때, 간선은 N(N-1)/2이 최대가 되겠네. 그러면 O(N^3)이고 N은 최대 500이니까 해결 가능이겠다.

#### 해설 본 후
* 너무 어렵게 생각했나... 그리고 스스로가 답이라고 생각한 방식에 확신이 없고 구체화를 못해서 금방 포기해버린 건 아닌가 싶다.
* 상위 강의 -> 하위 강의의 형태로 입력이 주어지므로, 위상 정렬 알고리즘 사용을 위해 이를 뒤집어줘야 한다. 생각은 했는데, 다른 좋은 방법이 있지 않을까 하며 시도를 안 해봤다...
* 특히 같은 차수에서 둘 다 선수 과목으로 하는 과목이 있을 경우 더 많은 시간이 걸리는 것을 채택해야 하는데 그 부분을 어떻게 구현할지 막막했다. 해설에서는 똑같이 위상정렬을 돌되, max()를 이용하여 그 부분을 체크해줬다.
* 모르겠으면 그냥 많이 풀어보는 게 장땡인 건 맞을 것 같다. 정진합시다.

In [2]:
# X

from collections import deque

# 듣고자 하는 강의 수 N 입력
N = int(input())


course_times = [0] * (N+1) # 각 강의의 소요 시간
graph = [[] for _ in range(N+1)]
indegrees = [0] * (N+1)

# 강의 시간 및 선수 강의 정보 입력
for i in range(1, N+1):
    time, *precourse = map(int, input().split())
    course_times[i] = time
    for p in precourse[:-1]:
        graph[p].append(i)
        indegrees[i] += 1

shortest_time = [0] * (N+1)

q = deque()

for i in range(1, N+1):
    if indegrees[i] == 0:
        q.append(i)
        
while q:
    course = q.popleft()
    

[0, 10, 10, 4, 4, 3]
[[], [], [1], [1], [3, 1], [3]]


#### 해설 코드 참고

In [ ]:
from collections import deque
import copy

# 노드 개수 입력
v = int(input())
# 모든 노드에 대한 진입차수를 0으로 초기화
indegree = [0] * (v+1)
# 각 노드에 연결된 간선 정보를 담기 위한 연결 리스트(그래프) 초기화
graph = [[] for i in range(v+1)]
# 각 강의 시간을 0으로 초기화
time = [0] * (v+1)

# 방향 그래프의 모든 간선 정보를 입력받기
for i in range(1, v+1):
    data = list(map(int, input().split()))
    time[i] = data[0] # 첫 번째 수는 시간 정보를 담고 있음
    for x in data[1:-1]:
        indegree[i] += 1
        graph[x].append(i) ## 여기도 키포인트
        
# 위상 정렬 함수
def topology_sort():
    result = copy.deepcopy(time) # 알고리즘 수행 결과를 담을 리스트
    q = deque() # 큐 기능을 위한 deque 라이브러리 사용
    
    # 처음 시작할 때는 진입차수가 0인 노드를 큐에 삽입
    for i in range(1, v + 1):
        if indegree[i] == 0:
            q.append(i)
            
    # 큐가 빌 때까지 반복
    while q:
        # 큐에서 원소 꺼내기
        now = q.popleft()
        # 해당 원소와 연결된 노드들의 진입차수에서 1 빼기
        for i in graph[now]:
            result[i] = max(result[i], result[now] + time[i]) ## 여기가 키포인트라고 생각한다.
            indegree[i] -= 1
            # 새롭게 진입차수가 0이 되는 노드를 큐에 삽입
            if indegree[i] == 0:
                q.append(i)
                
    # 위상 정렬 수행 결과 출력
    for i in range(1, v+1):
        print(result[i])

topology_sort()